<img src="https://raw.githubusercontent.com/euroargodev/argopy/master/docs/_static/argopy_logo_long.png" alt="argopy logo" width="200"/>

# Training Camp - Sept 22<sup>th</sup> 2025

***

## Notebook Title : Compute more variables from an Argo dataset

**Author contact : [G. Maze](https://annuaire.ifremer.fr/cv/17182)**

**Description:**

This notebook shows how to complement an existing Argo data dataset with more variables using argopy xarray extensions for:
- thermodynamic equations of seawater based on TEOS-10,
- optical modeling,
- the CANYON-MED neural network.

This is basically a notebook to explore this [section of the Argopy documentation](https://argopy.readthedocs.io/en/v1.3.0/user-guide/working-with-argo-data/data_computation.html).

*This notebook was developed with Argopy version: 1.3*

***

Let's start with the usual import:

In [ ]:
from argopy import DataFetcher

And to prevent cell output to be too large, we won't display xarray object attributes:

In [ ]:
import xarray as xr
xr.set_options(display_expand_attrs = False)

To demonstrate how to use argopy xarray extensions to compute more variables, we first need to load a dataset:

In [ ]:
f = DataFetcher().region([-75, -45, 20, 30, 0, 10, '2011-01', '2011-06'])
ds = f.data.copy()
ds.argo

### Compute TEOS-10 variables

You can compute additional ocean thermodynamic variables from [TEOS-10](https://teos-10.github.io/GSW-Python/intro.html) using the `teos10` argopy xarray extension like this:

In [ ]:
ds = ds.argo.teos10()
ds.data_vars

<br>

Where you can see the addition of new variables.

Note that each new variable comes with the appropriate attributes:

In [ ]:
ds['CT'].attrs

<br>

The default list of variables with the default call to `teos10()` is `SA`, `CT`, `SIG0`, `N2`, `PV` and `PTEMP`.

Other optional variables are `SOUND_SPEED` and `CNDC`.

To specify variables to compute, put them in a list:

In [ ]:
ds = f.data.copy()
ds.argo.teos10(['SA', 'CT']).data_vars

#### 🛟 NOTE

The `teos10` extension uses `PRES`, `TEMP` and `PSAL` to compute new variables. 

With the current version of argopy (1.3.0), there is no possibility to use ADJUSTED values. This may evolve in a future release.

#### ✏️ EXERCICE

Load data from float 6902746 and compute conductivity.

💡 Code hint:
```python
f = DataFetcher().float(...
```

In [ ]:
# Your code

### Compute optical modelling variables

The argopy `optic` xarray accessor extension provides methods to compute standard variables from optical modeling of the upper ocean. 
    
It can be used to:
- compute the depth of the euphotic zone, 
- compute the first optical depth, 
- compute the depth where PAR reaches some threshold value,
- search and qualify Deep Chlorophyll Maxima.

First, let's download some BGC-Argo float data with PAR variable. To do so, we need to use the `expert` mode:

In [ ]:
ds = DataFetcher(ds='bgc', mode='expert', params='DOWNWELLING_PAR').float(5907088).data
ds.argo

<br>

and since the `optic` extension only accept a collection of profiles, not points, we need to transform the dataset:

In [ ]:
dsp = ds.argo.point2profile()
dsp.argo

<br>

Using the following jupyter command will help you find the appropriate method:

In [ ]:
dsp.argo.optic?

<br>

Let's compute the depth of the euphotic zone:

In [ ]:
dsp = dsp.argo.optic.Zeu()
dsp['Zeu']

<br>

Note that new variable comes with the appropriate attributes:

In [ ]:
dsp['Zeu'].attrs

#### 🔍 Pro tip
You can plot the new variable along the float trajectory like this:

(easier methods shall be developped in the future)

In [ ]:
from argopy.plot import scatter_map
scatter_map(dsp, hue='Zeu', legend=False, cbar=True, cmap='Spectral_r');

#### ✏️ EXERCICE

Two methods, `percentage` and `KdPAR` are available to compute the euphotic zone depth. Compute and compare the two methods output with the same float 5907088 data.

It is also possible to load float data with an [ArgoFloat](https://argopy.readthedocs.io/en/v1.3.0/generated/argopy.ArgoFloat.html#argopy.ArgoFloat) instance.

💡 Code hint:
```python
from argopy import ArgoFloat
dsp = ArgoFloat(5907088).open_dataset(...
Zeu1 = dsp.argo.optic.Zeu(method=..., inplace=False)
```

In [ ]:
# Your code here

### Carbonate and nutrients in the Mediterranean Sea

For BGC data, it may be possible to complement a dataset with predictions of the water-column nutrient concentrations and carbonate system variables.

This is currently possible for data located in the Mediterranean Sea using the the CANYON-MED model. CANYON-MED is a Regional Neural Network Approach to Estimate Water-Column Nutrient Concentrations and Carbonate System Variables in the Mediterranean Sea ([Fourier et al, 2020](http://dx.doi.org/10.3389/fmars.2020.00620)). When using this method, please cite the paper.

This model is available in argopy as an extension to the argo xarray accessor. It can be used to predict PO4, NO3, DIC, SiOH4, AT and pHT.

For CANYON-MED to make prediction, only oxygen is required. So let's load some data:

In [ ]:
f = DataFetcher(ds='bgc', mode='standard', params='DOXY', measured='DOXY').float(1902605)
ds = f.data.copy()
ds.argo

<br>

We can then predict all possible variables:

In [ ]:
ds.argo.canyon_med.predict()
ds['DIC']

#### ✏️ EXERCICE

Re-load original data and predict only nutrients variables.

In [ ]:
# Your code here

### 
***
Useful argopy commands:
```python
argopy.reset_options()
argopy.show_options()
argopy.status()
argopy.clear_cache()
argopy.show_versions()
```
***
![logo](https://raw.githubusercontent.com/euroargodev/argopy-training/refs/heads/main/for_nb_producers/template_argopy_training_EAONE.png)